<a href="https://colab.research.google.com/github/theaidran/ollama_youtube_summarize/blob/main/ollama_youtube_summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# All credits to https://github.com/martinopiaggi/summarize
# Good to see links:
# https://github.com/hsiehjackson/RULER
# https://ollama.com/library
# https://amgadhasan.substack.com/p/sota-asr-tooling-long-form-transcription


#1 Choose source url
Link = "https://www.youtube.com/watch?v=UmSdw1Rf7XM" #@param {type:"string"}
Type_of_source = "Youtube video or playlist" #@param ['Youtube video or playlist', 'Google Drive video link','Dropbox video link', 'Local file']
Type = Type_of_source
URL = Link

#@markdown ---
#markdown Insert your API key depending on which endpoint you want to use
# API Configuration
api_key = "ollama"
api_endpoint = "Custom"
ollama_model = "llama3" # param ['llama3', 'phi3', 'vicuna:13b-v1.5-16k-q4_0']

endpoints = {
    "Groq": "https://api.groq.com/openai/v1",
    "OpenAI": "https://api.openai.com/v1",
    "Custom": "http://localhost:11434/v1"  # ollama endpoint
}
base_url = endpoints.get(api_endpoint)

models = {
    "Groq": "llama3-8b-8192",
    "OpenAI": "gpt-3.5-turbo",
    "Custom": ollama_model  # Placeholder for any custlom model
}
model = models.get(api_endpoint)

use_Youtube_captions = False #@param {type:"boolean"}


#2 launch ollama server with llama3
!curl -fsSL https://ollama.com/install.sh | sh
!tmux new -d ollama serve
!ollama run {ollama_model} !


#3 install libs: youtube and whisperx
import subprocess
import re
import os
import IPython
from IPython.display import clear_output

if use_Youtube_captions:
  !pip install youtube-transcript-api
  from youtube_transcript_api import YouTubeTranscriptApi

if (not Type == "Youtube video or playlist") or (not use_Youtube_captions):
  try:
    import whisperx
  except ImportError:
    !pip install git+https://github.com/m-bain/whisperx.git > /dev/null
    #clear_output()

try:
  import openai
except ImportError:
  !pip install openai
  #clear_output()

import openai
client = openai.OpenAI(api_key=api_key, base_url=base_url)


if Type == "Youtube video or playlist":
  try:
    import pytube
  except ImportError:
    !pip install git+https://github.com/pytube/pytube
    #clear_output()
    from pytube import YouTube

if Type == "Google Drive video link":
  from google.colab import drive
  drive.mount('/gdrive')

#4 Video fetching
skip_transcription=False
text = ""
textTimestamps = ""

def seconds_to_time_format(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}"

def download_youtube_audio_only(url):
    yt = YouTube(url)
    audio_stream = yt.streams.get_audio_only()
    saved_path = audio_stream.download(output_path=".", skip_existing=True)
    return saved_path

def download_youtube_captions(url):
    regex = r'(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|(?:v|e(?:mbed)?)\/|\S*?[?&]v=)|youtu\.be\/)([a-zA-Z0-9_-]{11})'
    video_id =  re.search(regex, url).group(1)
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

    try:
      transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
    except:
      for available_transcript in transcript_list:
        if available_transcript.is_translatable:
          transcript = available_transcript.translate('en').fetch()
          break

    text = ""
    for entry in transcript:
            start_time = seconds_to_time_format(entry['start'])
            text += f"{start_time} {entry['text'].strip()}\n"

    transcript_file_name = f"{video_id}_captions.md"

    with open(transcript_file_name, 'w', encoding='utf-8') as f:
      f.write(text)

    return text,transcript_file_name

if Type == "Youtube video or playlist":
    #clean youtube url from timestamp
    URL = re.sub('\&t=\d+s?', '', URL)
    if use_Youtube_captions:
      text, transcript_file_name = download_youtube_captions(URL)
      skip_transcription=True
    else:
      video_path_local =  download_youtube_audio_only(URL)

elif Type == "Google Drive video link":
  subprocess.run(['ffmpeg', '-y', '-i', '/gdrive/My Drive/' + URL, '-vn', '-acodec', 'pcm_s16le',
                  '-ar', '16000', '-ac', '1', 'gdrive_audio.wav'], check=True)
  video_path_local = "gdrive_audio.wav"

elif Type == "Dropbox video link":
    subprocess.run(['wget', URL, '-O', 'dropbox_video.mp4'], check=True)
    subprocess.run(['ffmpeg', '-y', '-i', 'dropbox_video.mp4', '-vn', '-acodec', 'pcm_s16le',
                    '-ar', '16000', '-ac', '1', 'dropbox_video_audio.wav'], check=True)
    video_path_local = "dropbox_video_audio.wav"

elif Type == "Local file":
    local_file_path = Source
    subprocess.run(['ffmpeg', '-y', '-i', local_file_path, '-vn', '-acodec', 'pcm_s16le',
                    '-ar', '16000', '-ac', '1', 'local_file_audio.wav'], check=True)
    video_path_local = "local_file_audio.wav"


#5 transcription using whisperx
if not skip_transcription:
  import whisperx
  language = "auto" # param {type:"string"}
  #initial_prompt = "" # @param {type:"string"}

  asr_options={"temperatures": 0, "beam_size": 1, "without_timestamps": True,}
  # Run on GPU with FP16
  modelWhisperX = whisperx.load_model("large-v2", device="cuda", compute_type="float16", asr_options=asr_options)

  transcription = modelWhisperX.transcribe(str(video_path_local), batch_size=16,
                                    language=None if language == "auto" else language,
                                    task="translate")

  transcript_file_name = os.path.splitext(video_path_local)[0]+'.md'

  with open(transcript_file_name, 'w') as f:
    for segment in transcription["segments"]:
        start_time = seconds_to_time_format(segment['start'])
        text += f"{start_time} {segment['text'].strip()} "

    f.write(text)


#6 Make a summary
prompt_type = "Summarization"  # param ['Summarization', 'Only grammar correction with highlights']
# markdown Set the number of parallel API calls (be mindful of usage rate limits)
parallel_api_calls = 1 # param


# Define your prompts using a dictionary for easier management
prompts = {
    'Summarization': """Summarize the video transcript excerpt including a concise title that reflects the content. Wrap the title with **markdown bold notation**. Write the summary as if you are continuing a conversation without needing to signal a beginning. Here is the transcript: """,
    'Only grammar correction with highlights': """Repeat the following text correcting any grammatical errors and formatting error. Highlight only the important quote (if there are any) with **markdown bold notation**. Focus solely on the essence of the content as if you are continuing a conversation without using any form of introduction like 'Here's the corrected text:'. Here is the text to fix: """
}

# Select the appropriate prompt
summary_prompt = prompts[prompt_type]


def extract_and_clean_timestamps(text_chunks):
    timestamp_pattern = re.compile(r'(\d{2}:\d{2}:\d{2})')
    cleaned_texts = []
    timestamp_ranges = []
    for chunk in text_chunks:
        timestamps = timestamp_pattern.findall(chunk)
        if timestamps:
            for timestamp in timestamps:
                # Remove each found timestamp from the chunk
                chunk = chunk.replace(timestamp, "")
            timestamp_ranges.append(timestamps[0])  # Assuming you want the first timestamp per chunk
        else:
            timestamp_ranges.append("")
        cleaned_texts.append(chunk.strip())  # Strip to remove any leading/trailing whitespace
    return cleaned_texts, timestamp_ranges

def format_timestamp_link(timestamp):
    if Type == "Youtube video or playlist":
      hours, minutes, seconds = map(int, timestamp.split(':'))
      total_seconds = hours * 3600 + minutes * 60 + seconds
      return f"{timestamp} - {URL}&t={total_seconds}"
    else:
      return f"{timestamp}"

import concurrent.futures
import time

def summarize(prompt):
    completion = client.chat.completions.create(
            model=model,
            messages=[
            {"role": "system", "content": summary_prompt},
            {"role": "user", "content": prompt}
            ],
            max_tokens=4096
    )
    return completion.choices[0].message.content

def process_and_summarize(text):
    chunk_size, overlap_size = 3072, 20
    texts = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size - overlap_size)]
    cleaned_texts, timestamp_ranges = extract_and_clean_timestamps(texts)
    summaries = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=parallel_api_calls) as executor:
        future_to_chunk = {executor.submit(summarize, text_chunk): idx for idx, text_chunk in enumerate(cleaned_texts)}
        for future in concurrent.futures.as_completed(future_to_chunk):
            idx = future_to_chunk[future]
            try:
                summarized_chunk = future.result()
                summary_piece = format_timestamp_link(timestamp_ranges[idx]) + " " + summarized_chunk
                summary_piece += "\n"
                summaries.append((idx, summary_piece))
            except Exception as exc:
                print(f'Chunk {idx} generated an exception: {exc}')
                # Resubmit the task with the new model
                time.sleep(10)
                future_to_chunk[executor.submit(summarize, texts[idx])] = idx

    summaries.sort()  # Ensure summaries are in the correct order
    global final_summary
    final_summary = "\n\n".join([summary for _, summary in summaries])

    # Save the final summary
    final_name = transcript_file_name.replace(".md", "_FINAL.md") if Type != "Dropbox video link" else "final_dropbox_video.md"
    with open(final_name, 'w') as f:
        f.write(final_summary)


process_and_summarize(text)
clear_output()
IPython.display.Markdown(final_summary)


00:00:00 - https://www.youtube.com/watch?v=UmSdw1Rf7XM&t=0 **Chat GPT Desktop App Now Available for Mac Users**

The summary continues... The new Chat GPT desktop app is out, and it's currently only available on Mac devices. To get started, search for "Chat GPT" in the app store, but be careful as there are fake apps trying to trick users into thinking they're the official one. The real deal can be found at chatgpt.com, where you'll need to log in and download the launcher. This app requires a Mac with Apple Silicon (M1, M2, or M3) running macOS 14 or later. The new launcher allows for voice conversations and screenshot sharing, making it easier to interact with Chat GPT on your desktop.


00:03:16 - https://www.youtube.com/watch?v=UmSdw1Rf7XM&t=196 **AI Desktop App Features: Hype or Reality?**

The video transcript excerpt discusses the features of ChatGPT's desktop app, specifically the claim of offline access and its limitations. The host attempts to use the app without an internet connection, but it fails to work in offline mode. This suggests that the feature is not yet implemented or may be hallucinated by the AI model. Additionally, the host highlights some of the best YouTube channels for staying up-to-date on AI news, including Two Minute Papers, Lex Fridman, and Matt Wolfe.


00:05:55 - https://www.youtube.com/watch?v=UmSdw1Rf7XM&t=355 **Hands-on with AI: Demos and Impacts**

The video continues discussing the potential of an AI-powered app that can read screens, allowing for seamless sharing and collaboration. The narrator showcases its capabilities by creating a design on their desktop, taking a screenshot, and asking the AI to code it into a website. The AI generates the code and creates a functional website with a joke popping up when a button is pressed. Although only available on Mac currently, plans are in place for Windows compatibility later this year.


00:08:22 - https://www.youtube.com/watch?v=UmSdw1Rf7XM&t=502 **Exploring Cross-Platform Chat Fun with PC and Mac** 

The speaker continues their conversation sharing their experience with a cross-platform chat tool that allows seamless communication between PC and Mac users. They mention being a mostly PC guy, but still enjoying the convenience of having a little chat window open all the time, accessible by pressing just two buttons. The video also touches on the benefits of using this technology for productivity and staying connected with others, regardless of their device preferences.
